In [1]:
import numpy as np
import pandas as pd

In [2]:
def serialize(df, name, how='columns'): open(f"{name}.js", 'w').write(f"var {name} = {df.to_json(orient=how)};\n")

## Differential Proteomics

In [8]:
diff_proteins = pd.read_excel('20170323_ALS_CTR_iMNs_protein_log2FC.xls')
diff_proteins.head()

,Uniprot,Protein,log2FC,score,FDR,log_oddsDE,Symbol
0,Q9NX47,MARH5_HUMAN,-0.148362,0.000435,0.317977,-7.74037,MARCH5
1,Q15019,SEPT2_HUMAN,-0.993235,1.000000,0.000000,184.09800,SEPT2
2,Q9UH03,SEPT3_HUMAN,-0.785533,1.000000,0.000000,53.96050,SEPT3
3,Q99719,SEPT5_HUMAN,0.118518,0.000540,0.313857,-7.52392,SEPT5
4,Q14141,SEPT6_HUMAN,-0.619176,0.996636,0.000119,5.69116,SEPT6


In [9]:
diff_proteins = diff_proteins.set_index('Symbol')[['log2FC', 'FDR', 'score', 'log_oddsDE']]
diff_proteins = diff_proteins.replace(' ', np.nan)
diff_proteins = diff_proteins.rename(columns={'log2FC':'logFC', 'FDR':'q'})
diff_proteins = diff_proteins.dropna(how='any', subset=['logFC','q'])
diff_proteins = diff_proteins[~diff_proteins.index.duplicated(keep='first')]


diff_proteins['q'] = -np.log10(diff_proteins['q'])
human_max_finite_log_qVal = np.max(diff_proteins['q'][np.isfinite(diff_proteins['q'])])
diff_proteins['q'] = np.around(np.clip(diff_proteins['q'], 0, human_max_finite_log_qVal), decimals=2)
print("max q-val: " + str(human_max_finite_log_qVal))

diff_proteins = diff_proteins.round(2)
diff_proteins.head()

max q-val: 18.42829116819131


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log10
  


,logFC,q,score,log_oddsDE
Symbol,,,,
MARCH5,-0.15,0.50,0.0,-7.74
SEPT2,-0.99,18.43,1.0,184.10
SEPT3,-0.79,18.43,1.0,53.96
SEPT5,0.12,0.50,0.0,-7.52
SEPT6,-0.62,3.92,1.0,5.69


In [10]:
serialize(diff_proteins[['logFC','q']], 'diff_proteins', how='index')

## Differential Transcriptomics

In [24]:
diff_transcripts = pd.read_csv('ALSvsCtrl_DEseq_iMN_18JAN2016_converted_symbols.csv')
diff_transcripts.head()

,Symbol,ID,Exp Log Ratio,Exp p-value,Exp False Discovery Rate (q-value),Entrez Gene Name
0,A1BG,ENSG00000121410,-0.168,6.08E-01,9.19E-01,alpha-1-B glycoprotein
1,A1BG-AS1,ENSG00000268895,0.348,2.65E-01,7.36E-01,A1BG antisense RNA 1
2,A1CF,ENSG00000148584,-0.159,8.42E-01,,APOBEC1 complementation factor
3,A2M,ENSG00000175899,-0.259,6.85E-01,9.44E-01,alpha-2-macroglobulin
4,A2M-AS1,ENSG00000245105,-1.151,8.47E-02,4.80E-01,A2M antisense RNA 1 (head to head)


In [25]:
diff_transcripts = diff_transcripts[['Symbol', 'Exp Log Ratio', 'Exp p-value', 'Exp False Discovery Rate (q-value)']]
diff_transcripts = diff_transcripts.rename(columns={'Exp Log Ratio':'logFC', 'Exp p-value':'p', 'Exp False Discovery Rate (q-value)':'q'})
diff_transcripts = diff_transcripts.replace(' ', np.nan)
diff_transcripts = diff_transcripts.dropna(how='all', subset=['q'])
diff_transcripts = diff_transcripts.set_index('Symbol')
diff_transcripts = diff_transcripts[~diff_transcripts.index.duplicated(keep='first')]

diff_transcripts.logFC = pd.to_numeric(diff_transcripts.logFC)
diff_transcripts.p = pd.to_numeric(diff_transcripts.p)
diff_transcripts.q = pd.to_numeric(diff_transcripts.q)


diff_transcripts['q'] = -np.log10(diff_transcripts['q'])
human_max_finite_log_qVal = np.max(diff_transcripts['q'][np.isfinite(diff_transcripts['q'])])
diff_transcripts['q'] = np.around(np.clip(diff_transcripts['q'], 0, human_max_finite_log_qVal), decimals=2)
print("max q-val: " + str(human_max_finite_log_qVal))

diff_transcripts = diff_transcripts.round(2)
diff_transcripts.head()

max q-val: 30.221848749616356


,logFC,p,q
Symbol,,,
A1BG,-0.17,0.61,0.04
A1BG-AS1,0.35,0.26,0.13
A2M,-0.26,0.68,0.03
A2M-AS1,-1.15,0.08,0.32
A2ML1,-1.98,0.01,0.74


In [26]:
serialize(diff_transcripts[['logFC','q']], 'diff_transcripts', how='index')

## Proteomics

In [5]:
protein = pd.read_csv('./protein_abundances_JL.tsv', sep='\t')
protein.head()

,Symbol,28iALS_1,28iALS_2,29iALS_1,29iALS_2,30iALS_1,52iALS_1,52iALS_2,00iCTR_1,00iCTR_2,25iCTR_1_A,25iCTR_1_B,25iCTR_2,83iCTR_1,83iCTR_2
0,UBA6,1515.96,1325.49,2046.09,2373.03,3139.98,1888.97,1133.23,2428.80,2369.05,3298.28,3562.29,4558.27,2698.67,2410.73
1,ESYT2,0.00,0.00,166.46,245.38,140.69,0.00,0.00,0.00,0.00,88.14,81.26,0.00,92.94,24.49
2,SHTN1,0.00,0.00,0.00,64.01,206.67,0.00,179.03,93.06,70.56,329.18,274.13,758.07,1469.21,1535.94
3,GXYLT2,45.16,0.00,220.66,135.43,0.00,556.94,390.97,40.96,37.02,22.63,0.00,0.00,0.00,25.72
4,ILVBL,0.00,200.78,670.33,765.68,493.34,426.52,122.09,355.28,155.03,690.57,623.43,1104.38,645.16,752.21


In [6]:
protein[protein.duplicated('Symbol', keep=False)]

,Symbol,28iALS_1,28iALS_2,29iALS_1,29iALS_2,30iALS_1,52iALS_1,52iALS_2,00iCTR_1,00iCTR_2,25iCTR_1_A,25iCTR_1_B,25iCTR_2,83iCTR_1,83iCTR_2
1142,TMPO,97.56,277.71,421.52,516.68,622.26,915.61,492.38,821.63,743.06,796.69,604.21,646.67,616.47,514.38
1143,TMPO,11056.10,15455.20,19644.70,18770.50,12320.40,22956.80,13455.50,38782.20,31021.30,27139.30,27785.90,26149.80,21507.10,13254.50


In [7]:
protein = protein.drop(protein.index[[1142]])
protein.loc[1141:1144]

,Symbol,28iALS_1,28iALS_2,29iALS_1,29iALS_2,30iALS_1,52iALS_1,52iALS_2,00iCTR_1,00iCTR_2,25iCTR_1_A,25iCTR_1_B,25iCTR_2,83iCTR_1,83iCTR_2
1141,ECI1,417.55,1708.57,4864.76,4880.71,3327.45,2309.01,700.29,3516.64,2464.87,5150.95,5255.14,5286.32,3024.46,3124.55
1143,TMPO,11056.10,15455.20,19644.70,18770.50,12320.40,22956.80,13455.50,38782.20,31021.30,27139.30,27785.90,26149.80,21507.10,13254.50
1144,STAT1,511.31,700.82,2181.72,2205.62,3070.15,1426.33,666.01,800.58,479.76,3673.22,3546.43,4603.48,1832.12,1887.76


In [8]:
protein = protein.set_index('Symbol')
protein = protein.round(2)
protein.head()

,28iALS_1,28iALS_2,29iALS_1,29iALS_2,30iALS_1,52iALS_1,52iALS_2,00iCTR_1,00iCTR_2,25iCTR_1_A,25iCTR_1_B,25iCTR_2,83iCTR_1,83iCTR_2
Symbol,,,,,,,,,,,,,,
UBA6,1515.96,1325.49,2046.09,2373.03,3139.98,1888.97,1133.23,2428.80,2369.05,3298.28,3562.29,4558.27,2698.67,2410.73
ESYT2,0.00,0.00,166.46,245.38,140.69,0.00,0.00,0.00,0.00,88.14,81.26,0.00,92.94,24.49
SHTN1,0.00,0.00,0.00,64.01,206.67,0.00,179.03,93.06,70.56,329.18,274.13,758.07,1469.21,1535.94
GXYLT2,45.16,0.00,220.66,135.43,0.00,556.94,390.97,40.96,37.02,22.63,0.00,0.00,0.00,25.72
ILVBL,0.00,200.78,670.33,765.68,493.34,426.52,122.09,355.28,155.03,690.57,623.43,1104.38,645.16,752.21


In [9]:
serialize(protein, 'protein')

## Transcriptomics

In [10]:
transcripts = pd.read_csv('./rna_abundances_JL.tsv', sep='\t')
transcripts.head()

,symbol,00iCTR_1,25iCTR_1,83iCTR_1,28iALS_1,29iALS_1,30iALS_1,52iALS_1,00iCTR_2,25iCTR_2,83iCTR_2,28iALS_2,29iALS_2,30iALS_2,52iALS_2
0,TSPAN6,15.07520,42.212200,31.045900,23.438800,18.67430,23.518300,15.95140,18.99350,50.810600,37.202600,19.376000,19.72610,24.37710,16.93510
1,TNMD,3.46409,0.180462,0.209293,0.379753,1.38633,0.097987,7.19405,5.31750,0.152618,0.322817,0.603215,1.69970,0.12456,7.99093
2,DPM1,14.45850,15.473800,16.354500,12.372100,24.01650,17.426000,22.02340,18.79230,15.512700,17.521900,11.414600,24.04870,15.75790,20.58920
3,SCYL3,3.62369,2.468500,2.895030,3.202920,2.35784,2.196690,2.45420,3.91679,3.179870,2.835270,2.731020,2.44931,2.30431,2.35572
4,C1orf112,4.55001,2.773590,2.880550,1.738930,2.35351,4.300850,2.57059,6.02910,2.752660,3.493480,1.977490,2.68149,4.26484,3.07295


In [11]:
transcripts[transcripts.duplicated('symbol', keep=False)].head(15)

,symbol,00iCTR_1,25iCTR_1,83iCTR_1,28iALS_1,29iALS_1,30iALS_1,52iALS_1,00iCTR_2,25iCTR_2,83iCTR_2,28iALS_2,29iALS_2,30iALS_2,52iALS_2
119,KRT33A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
137,AGK,6.652920,6.782470,6.705490,8.498440,7.175460,7.463480,6.283780,8.401210,7.826070,7.477400,8.242860,7.731610,6.803810,6.301910
728,SYNE2,12.543700,9.219550,20.321400,14.236600,6.345310,9.832820,6.655190,11.611200,10.974100,20.879800,13.194900,5.272600,10.188500,7.366990
729,SYNE2,0.125712,0.000000,0.055844,0.075691,0.000000,0.122600,0.000000,0.000000,0.000000,0.060200,0.057673,0.051430,0.140259,0.050605
762,KIFC1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
835,CDH3,0.062083,0.107110,0.055234,0.073313,0.102287,0.057206,0.000000,0.057683,0.086164,0.000000,0.112172,0.142941,0.086770,0.000000
836,CDH3,1.898660,1.537070,2.943540,3.186920,4.930680,3.310170,3.410860,2.089190,1.508150,2.448410,2.681300,4.907780,3.468020,3.070170
970,ZNRD1,9.723120,8.069770,9.221410,7.678770,10.092100,9.160190,10.551000,12.398200,10.866900,11.385100,6.946990,9.904930,8.114340,12.081600
1046,ATP11A,4.908960,2.765230,3.984570,3.117400,2.584590,2.810190,3.331990,4.270140,3.466180,3.987330,2.569060,2.520620,2.787430,3.948490
1047,ATP11A,0.101993,0.000000,0.000000,0.000000,0.000000,0.000000,0.103436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.234438


In [12]:
transcripts = transcripts.groupby('symbol').sum(axis=1)

In [13]:
transcripts = transcripts.round(2)

In [14]:
serialize(transcripts, 'transcripts')

## Metadata

In [15]:
set(transcripts.columns)

{'00iCTR_1',
 '00iCTR_2',
 '25iCTR_1',
 '25iCTR_2',
 '28iALS_1',
 '28iALS_2',
 '29iALS_1',
 '29iALS_2',
 '30iALS_1',
 '30iALS_2',
 '52iALS_1',
 '52iALS_2',
 '83iCTR_1',
 '83iCTR_2'}

In [16]:
set(protein.columns)

{'00iCTR_1',
 '00iCTR_2',
 '25iCTR_1_A',
 '25iCTR_1_B',
 '25iCTR_2',
 '28iALS_1',
 '28iALS_2',
 '29iALS_1',
 '29iALS_2',
 '30iALS_1',
 '52iALS_1',
 '52iALS_2',
 '83iCTR_1',
 '83iCTR_2'}

In [17]:
set(transcripts.columns) - set(protein.columns)

{'25iCTR_1', '30iALS_2'}

In [18]:
set(protein.columns) - set(transcripts.columns)

{'25iCTR_1_A', '25iCTR_1_B'}

In [27]:
transcriptomic_classes = pd.DataFrame(transcripts.columns.str.split('i').str[1].str.split('_').str[0], index=transcripts.columns, columns=['condition'])
transcriptomic_classes

,condition
00iCTR_1,CTR
25iCTR_1,CTR
83iCTR_1,CTR
28iALS_1,ALS
29iALS_1,ALS
30iALS_1,ALS
52iALS_1,ALS
00iCTR_2,CTR
25iCTR_2,CTR
83iCTR_2,CTR


In [28]:
proteomic_classes = pd.DataFrame(protein.columns.str.split('i').str[1].str.split('_').str[0], index=protein.columns, columns=['condition'])
proteomic_classes

,condition
28iALS_1,ALS
28iALS_2,ALS
29iALS_1,ALS
29iALS_2,ALS
30iALS_1,ALS
52iALS_1,ALS
52iALS_2,ALS
00iCTR_1,CTR
00iCTR_2,CTR
25iCTR_1_A,CTR


In [30]:
serialize(transcriptomic_classes, 'transcriptomic_classes', how='index')

In [31]:
serialize(proteomic_classes, 'proteomic_classes', how='index')